## Introduction

This notebook demonstrates how to extract entities and relationships from documents using modern NLP techniques and LLMs. You'll learn to:
- Use LLMs for entity and relationship extraction
- Process document collections systematically
- Handle different document types and formats
- Validate and clean extracted knowledge
- Prepare data for knowledge graph construction

**## Prerequisites and Setup**

In [12]:
# Install and import all packages in one cell
!pip install -q openai langchain langchain-openai spacy pandas matplotlib seaborn nltk tiktoken

# Download spacy model
!python -m spacy download en_core_web_sm

# Import packages immediately after installation
import importlib
import sys

# Force reload of modules if already imported
if 'langchain_openai' in sys.modules:
    importlib.reload(sys.modules['langchain_openai'])

# Import all required libraries
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
from typing import List, Dict, Any, Tuple

# Import with error handling
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
    print("✅ spaCy loaded successfully")
except Exception as e:
    print(f"⚠️ spaCy issue: {e}")

try:
    import nltk
    nltk.download('punkt_tab', quiet=True)  # Updated for newer NLTK versions
    nltk.download('punkt', quiet=True)      # Fallback for older versions
    nltk.download('stopwords', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    print("✅ NLTK loaded successfully")
except Exception as e:
    print(f"⚠️ NLTK issue: {e}")

try:
    from langchain_openai import ChatOpenAI
    from langchain.prompts import PromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langchain.schema import Document
    print("✅ LangChain loaded successfully")
except Exception as e:
    print(f"⚠️ LangChain issue: {e}")

try:
    import tiktoken
    print("✅ tiktoken loaded successfully")
except Exception as e:
    print(f"⚠️ tiktoken issue: {e}")

# Set up OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual key

# Initialize LLM with error handling
try:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    print("✅ OpenAI LLM initialized successfully")
except Exception as e:
    print(f"⚠️ OpenAI initialization issue: {e}")
    print("Please check your API key is set correctly")

print("\n🚀 Setup complete! All packages loaded in one cell.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ spaCy loaded successfully
✅ NLTK loaded successfully
✅ LangChain loaded successfully
✅ tiktoken loaded successfully
✅ OpenAI LLM initialized successfully

🚀 Setup complete! All packages loaded in one cell.


**## Part 1: Understanding Entity and Relationship Extraction**

In [ ]:
### Sample Document Collection

# Sample research paper abstracts for demonstration
sample_documents = [
    {
        "id": "paper_1",
        "title": "Attention Is All You Need",
        "abstract": """We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show that these models are superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU points. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight P100 GPUs, a small fraction of the training costs of the best models from the literature.""",
        "authors": ["Ashish Vaswani", "Noam Shazeer", "Niki Parmar", "Jakob Uszkoreit"],
        "year": 2017,
        "venue": "NIPS"
    },
    {
        "id": "paper_2",
        "title": "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding",
        "abstract": """We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models, BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial task-specific architecture modifications.""",
        "authors": ["Jacob Devlin", "Ming-Wei Chang", "Kenton Lee", "Kristina Toutanova"],
        "year": 2018,
        "venue": "NAACL"
    },
    {
        "id": "paper_3",
        "title": "GPT-3: Language Models are Few-Shot Learners",
        "abstract": """Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. However, this paradigm requires task-specific fine-tuning datasets of thousands or tens of thousands of examples, and still often fails to match human performance. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language model, and test its performance in the few-shot setting.""",
        "authors": ["Tom B. Brown", "Benjamin Mann", "Nick Ryder", "Melanie Subbiah"],
        "year": 2020,
        "venue": "NeurIPS"
    }
]

print(f"📚 Loaded {len(sample_documents)} sample documents")
for doc in sample_documents:
    print(f"   • {doc['title']} ({doc['year']})")

**### Document Processing Utilities**

In [ ]:
class DocumentProcessor:
    """Utility class for processing and analyzing documents."""

    def __init__(self):
        self.encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    def count_tokens(self, text: str) -> int:
        """Count tokens in text for LLM processing."""
        return len(self.encoding.encode(text))

    def chunk_text(self, text: str, max_tokens: int = 1000, overlap: int = 100) -> List[str]:
        """Split text into chunks with specified token limits."""
        tokens = self.encoding.encode(text)
        chunks = []

        start = 0
        while start < len(tokens):
            end = min(start + max_tokens, len(tokens))
            chunk_tokens = tokens[start:end]
            chunk_text = self.encoding.decode(chunk_tokens)
            chunks.append(chunk_text)

            if end >= len(tokens):
                break
            start = end - overlap

        return chunks

    def clean_text(self, text: str) -> str:
        """Clean and normalize text for processing."""
        # Remove excessive whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove special characters but keep punctuation
        text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
        return text.strip()

    def extract_sentences(self, text: str) -> List[str]:
        """Extract sentences from text using NLTK."""
        sentences = nltk.sent_tokenize(text)
        return [s.strip() for s in sentences if len(s.strip()) > 10]

# Initialize processor
doc_processor = DocumentProcessor()

# Demonstrate text processing
sample_text = sample_documents[0]["abstract"]
print("📊 Document Processing Analysis:")
print(f"Token count: {doc_processor.count_tokens(sample_text)}")
print(f"Sentence count: {len(doc_processor.extract_sentences(sample_text))}")
print(f"First sentence: {doc_processor.extract_sentences(sample_text)[0]}")

**## Part 2: LLM-Based Entity Extraction**

In [ ]:
### Entity Extraction with Prompts

class EntityExtractor:
    """Extract entities from text using LLM prompting."""

    def __init__(self, llm):
        self.llm = llm
        self.entity_prompt = PromptTemplate.from_template("""
You are an expert at extracting entities from academic research papers.
Extract the following types of entities from the given text:

ENTITY TYPES:
- PERSON: Names of researchers, authors, scientists
- ORGANIZATION: Universities, companies, research labs
- CONCEPT: Technical concepts, methods, algorithms
- DATASET: Named datasets used in research
- METRIC: Performance metrics, evaluation measures
- TECHNOLOGY: Software, hardware, tools

TEXT:
{text}

INSTRUCTIONS:
1. Extract entities exactly as they appear in the text
2. Classify each entity with one of the types above
3. Return results in JSON format
4. Only extract entities that are clearly mentioned

OUTPUT FORMAT:
{
  "entities": [
    {"text": "entity name", "type": "ENTITY_TYPE", "context": "surrounding context"},
    ...
  ]
}

JSON OUTPUT:
""")

    def extract_entities(self, text: str) -> Dict[str, Any]:
        """Extract entities from text using LLM."""
        try:
            # Create and execute the prompt
            chain = self.entity_prompt | self.llm | StrOutputParser()
            result = chain.invoke({"text": text})

            # Parse JSON response
            try:
                entities = json.loads(result)
                return entities
            except json.JSONDecodeError:
                # Handle cases where LLM doesn't return valid JSON
                print("⚠️ JSON parsing failed, attempting to extract entities from text")
                return self._parse_text_response(result)

        except Exception as e:
            print(f"❌ Error in entity extraction: {e}")
            return {"entities": []}

    def _parse_text_response(self, response: str) -> Dict[str, Any]:
        """Parse non-JSON responses from LLM."""
        # Simple fallback parsing - in practice, you'd want more robust parsing
        entities = []
        lines = response.split('\n')

        for line in lines:
            if ':' in line and any(entity_type in line.upper() for entity_type in
                                 ['PERSON', 'ORGANIZATION', 'CONCEPT', 'DATASET', 'METRIC', 'TECHNOLOGY']):
                try:
                    # Extract entity information from line
                    parts = line.split(':', 1)
                    if len(parts) == 2:
                        entity_info = parts[1].strip()
                        # This is a simplified parser - extend based on your needs
                        entities.append({
                            "text": entity_info,
                            "type": "CONCEPT",  # Default type
                            "context": line
                        })
                except:
                    continue

        return {"entities": entities}

    def process_document(self, document: Dict[str, Any]) -> Dict[str, Any]:
        """Process an entire document and extract entities."""
        # Combine title and abstract for processing
        text = f"Title: {document['title']}\n\nAbstract: {document['abstract']}"

        # Extract entities
        entities = self.extract_entities(text)

        # Add document metadata
        entities['document_id'] = document['id']
        entities['document_title'] = document['title']

        return entities

# Initialize entity extractor
entity_extractor = EntityExtractor(llm)

# Extract entities from first document
print("🔍 Extracting entities from sample document...")
sample_entities = entity_extractor.process_document(sample_documents[0])

print(f"✅ Extracted {len(sample_entities.get('entities', []))} entities")
for entity in sample_entities.get('entities', [])[:5]:  # Show first 5
    print(f"   • {entity.get('text', 'Unknown')} ({entity.get('type', 'Unknown')})")

**## Part 3: Relationship Extraction**

In [9]:
class RelationshipExtractor:
    """Extract relationships between entities using LLM prompting."""

    def __init__(self, llm):
        self.llm = llm
        self.relationship_prompt = PromptTemplate.from_template("""
You are an expert at extracting relationships from academic research papers.
Given the text and a list of entities, identify relationships between these entities.

RELATIONSHIP TYPES:
- AUTHORED: Person authored/wrote paper
- AFFILIATED_WITH: Person affiliated with organization
- BASED_ON: Concept/method based on another concept
- IMPROVES_ON: Method improves on previous method
- USES: Method/system uses technology/dataset
- EVALUATES_ON: Method evaluated on dataset
- ACHIEVES: Method achieves specific metric/performance
- CITES: Paper references another work
- INTRODUCES: Paper introduces new concept/method

TEXT:
{text}

ENTITIES:
{entities}

INSTRUCTIONS:
1. Only extract relationships that are clearly stated in the text
2. Use entities from the provided list
3. Specify relationship type, source entity, target entity
4. Include confidence score (0.0-1.0)
5. Return results in JSON format

OUTPUT FORMAT:
{
  "relationships": [
    {
      "source": "source entity",
      "target": "target entity",
      "relationship": "RELATIONSHIP_TYPE",
      "confidence": 0.9,
      "evidence": "text snippet supporting this relationship"
    },
    ...
  ]
}

JSON OUTPUT:
""")

    def extract_relationships(self, text: str, entities: List[Dict]) -> Dict[str, Any]:
        """Extract relationships from text given entities."""
        try:
            # Format entities for the prompt
            entity_list = []
            for entity in entities:
                entity_list.append(f"- {entity['text']} ({entity['type']})")
            entities_text = "\n".join(entity_list)

            # Create and execute the prompt
            chain = self.relationship_prompt | self.llm | StrOutputParser()
            result = chain.invoke({
                "text": text,
                "entities": entities_text
            })

            # Parse JSON response
            try:
                relationships = json.loads(result)
                return relationships
            except json.JSONDecodeError:
                print("⚠️ JSON parsing failed for relationships")
                return self._parse_relationship_text(result)

        except Exception as e:
            print(f"❌ Error in relationship extraction: {e}")
            return {"relationships": []}

    def _parse_relationship_text(self, response: str) -> Dict[str, Any]:
        """Parse non-JSON relationship responses."""
        relationships = []
        # Simple fallback parsing - extend based on your needs
        lines = response.split('\n')

        for line in lines:
            if '->' in line or 'relationship' in line.lower():
                # This is a simplified parser
                relationships.append({
                    "source": "extracted_entity",
                    "target": "extracted_entity",
                    "relationship": "UNKNOWN",
                    "confidence": 0.5,
                    "evidence": line.strip()
                })

        return {"relationships": relationships}

# Initialize relationship extractor
relationship_extractor = RelationshipExtractor(llm)

# Extract relationships for the first document
if sample_entities:
    first_doc_entities = sample_entities.get('entities', [])
    if first_doc_entities:
        print("🔗 Extracting relationships from sample document...")
        sample_relationships = relationship_extractor.extract_relationships(
            f"Title: {sample_documents[0]['title']}\n\nAbstract: {sample_documents[0]['abstract']}",
            first_doc_entities
        )

        print(f"✅ Extracted {len(sample_relationships.get('relationships', []))} relationships")
        for rel in sample_relationships.get('relationships', [])[:3]:  # Show first 3
            print(f"   • {rel.get('source', '')} --[{rel.get('relationship', '')}]--> {rel.get('target', '')}")

**## Part 4: Complete Knowledge Extraction Pipeline**

In [ ]:
class KnowledgeExtractor:
    """Combined entity and relationship extraction for documents."""

    def __init__(self, llm):
        self.entity_extractor = EntityExtractor(llm)
        self.relationship_extractor = RelationshipExtractor(llm)
        self.doc_processor = DocumentProcessor()

    def extract_knowledge(self, document: Dict[str, Any]) -> Dict[str, Any]:
        """Extract complete knowledge (entities + relationships) from a document."""

        print(f"🔍 Processing: {document['title'][:50]}...")

        # Step 1: Extract entities
        entity_result = self.entity_extractor.process_document(document)
        entities = entity_result.get('entities', [])

        # Step 2: Extract relationships
        text = f"Title: {document['title']}\n\nAbstract: {document['abstract']}"
        relationship_result = self.relationship_extractor.extract_relationships(text, entities)
        relationships = relationship_result.get('relationships', [])

        # Step 3: Compile complete knowledge structure
        knowledge = {
            'document_id': document['id'],
            'document_title': document['title'],
            'document_metadata': {
                'authors': document.get('authors', []),
                'year': document.get('year'),
                'venue': document.get('venue')
            },
            'entities': entities,
            'relationships': relationships,
            'extraction_stats': {
                'entity_count': len(entities),
                'relationship_count': len(relationships),
                'token_count': self.doc_processor.count_tokens(document['abstract'])
            }
        }

        return knowledge

# Initialize comprehensive knowledge extractor
knowledge_extractor = KnowledgeExtractor(llm)

# Extract knowledge from all sample documents
print("🚀 Starting comprehensive knowledge extraction...")
extracted_knowledge = []

for doc in sample_documents:
    try:
        knowledge = knowledge_extractor.extract_knowledge(doc)
        extracted_knowledge.append(knowledge)
    except Exception as e:
        print(f"❌ Error processing {doc['id']}: {e}")

print(f"\n📊 Knowledge Extraction Complete!")
print(f"   Documents processed: {len(extracted_knowledge)}")

# Summary statistics
total_entities = sum(len(doc.get('entities', [])) for doc in extracted_knowledge)
total_relationships = sum(len(doc.get('relationships', [])) for doc in extracted_knowledge)

print(f"   Total entities: {total_entities}")
print(f"   Total relationships: {total_relationships}")

**## Part 5: Export for Graph Construction**

In [ ]:
class KnowledgeExporter:
    """Export extracted knowledge for graph construction."""

    def to_graph_format(self, extracted_knowledge: List[Dict]) -> Dict[str, Any]:
        """Convert extracted knowledge to graph-ready format."""

        nodes = []
        edges = []
        node_id_map = {}

        for knowledge in extracted_knowledge:
            doc_id = knowledge['document_id']

            # Add document node
            doc_node_id = f"doc_{doc_id}"
            nodes.append({
                'id': doc_node_id,
                'label': knowledge['document_title'],
                'type': 'DOCUMENT',
                'properties': knowledge.get('document_metadata', {})
            })

            # Add entity nodes
            for entity in knowledge.get('entities', []):
                entity_text = entity['text']
                entity_id = f"entity_{hash(entity_text) % 100000}"

                if entity_id not in node_id_map:
                    nodes.append({
                        'id': entity_id,
                        'label': entity_text,
                        'type': entity['type'],
                        'properties': {
                            'context': entity.get('context', ''),
                            'source_document': doc_id
                        }
                    })
                    node_id_map[entity_id] = entity_text

                # Add document-entity relationship
                edges.append({
                    'source': doc_node_id,
                    'target': entity_id,
                    'relationship': 'CONTAINS',
                    'properties': {}
                })

            # Add relationship edges
            for relationship in knowledge.get('relationships', []):
                source_text = relationship.get('source', '')
                target_text = relationship.get('target', '')

                # Find corresponding node IDs
                source_id = None
                target_id = None

                for node_id, node_text in node_id_map.items():
                    if source_text == node_text:
                        source_id = node_id
                    if target_text == node_text:
                        target_id = node_id

                if source_id and target_id:
                    edges.append({
                        'source': source_id,
                        'target': target_id,
                        'relationship': relationship.get('relationship', 'RELATED'),
                        'properties': {
                            'confidence': relationship.get('confidence', 0.5),
                            'evidence': relationship.get('evidence', ''),
                            'source_document': doc_id
                        }
                    })

        return {
            'nodes': nodes,
            'edges': edges,
            'metadata': {
                'total_nodes': len(nodes),
                'total_edges': len(edges),
                'extraction_timestamp': pd.Timestamp.now().isoformat()
            }
        }

    def save_for_next_notebook(self, extracted_knowledge: List[Dict]):
        """Save processed data for graph construction notebook."""

        graph_data = self.to_graph_format(extracted_knowledge)

        final_export = {
            'extracted_knowledge': extracted_knowledge,
            'graph_data': graph_data,
            'extraction_metadata': {
                'extraction_date': pd.Timestamp.now().isoformat(),
                'source_documents': len(sample_documents),
                'total_entities': sum(len(k.get('entities', [])) for k in extracted_knowledge),
                'total_relationships': sum(len(k.get('relationships', [])) for k in extracted_knowledge)
            }
        }

        # Save to JSON for next notebook
        with open('processed_knowledge_for_graph.json', 'w') as f:
            json.dump(final_export, f, indent=2)

        print("✅ Saved processed knowledge to 'processed_knowledge_for_graph.json'")
        return final_export

# Export data for next notebook
exporter = KnowledgeExporter()
final_data = exporter.save_for_next_notebook(extracted_knowledge)

print(f"\n📋 Ready for Graph Construction:")
print(f"   • {final_data['extraction_metadata']['total_entities']} entities extracted")
print(f"   • {final_data['extraction_metadata']['total_relationships']} relationships identified")
print(f"   • Data saved for Notebook 12.3.2 (Graph Construction)")

print(f"\n🎯 Next Steps:")
print("   1. Load this data in Notebook 12.3.2")
print("   2. Create Neo4j nodes and relationships")
print("   3. Implement entity linking and deduplication")
print("   4. Build complete knowledge graph")

## Key Achievements

✅ **Automated Knowledge Extraction**: LLM-powered extraction of entities and relationships
✅ **Structured Output**: Well-formatted data ready for graph construction  
✅ **Quality Processing**: Error handling and validation throughout the pipeline
✅ **Scalable Approach**: Designed to handle large document collections
✅ **Export Ready**: Multiple formats for downstream graph construction

Continue to Notebook 12.3.2 to transform this extracted knowledge into a complete knowledge graph!